# Option Pricing with Deep Learning
**End-Term Project Implementation**

This notebook compares the **Black-Scholes model** with a **Neural Network (MLP)** for pricing options using historical volatility and contract features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

## 1. Load Dataset

In [ ]:
df = pd.read_excel('ASIANPAINT_Dataset.xlsx')
df['T_years'] = df['t'] / 365
df.head()

## 2. Black-Scholes Baseline Model

In [ ]:
S = df['underlying_value']
K = df['strike_price']
T = df['T_years']
r = df['r']
sigma = df['sigma']

d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)
bs_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

mse_bs = mean_squared_error(df['close'], bs_price)
mse_bs

## 3. Neural Network (MLP) Model

In [ ]:
X = df[['underlying_value','strike_price','T_years','r','sigma']]
y = df['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = MLPRegressor(hidden_layer_sizes=(128,128,128), activation='relu', solver='adam', max_iter=200, random_state=42)
mlp.fit(X_train, y_train)

pred_nn = mlp.predict(X_test)
mse_nn = mean_squared_error(y_test, pred_nn)
mse_nn

## 4. Model Comparison

In [ ]:
print('Black-Scholes MSE:', mse_bs)
print('Neural Network MSE:', mse_nn)

## 5. Prediction Visualization

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(y_test.values[:100], label='True Price')
plt.plot(pred_nn[:100], label='NN Prediction')
plt.legend()
plt.title('Neural Network vs True Option Prices')
plt.show()